Code Picked Retriever
---

## 環境構築

ライブラリのインストール。

In [ ]:
%%capture
%pip install langchain openai tiktoken

環境変数 `MY_OPENAI_API_KEY` に使用する API キーを設定しておきましょう。

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("MY_OPENAI_API_KEY")

## 関数

### get_knowledge_string

In [ ]:
def get_knowledge_string(root_dir, target_exts=[], ignore_paths=[]):
    """
    root_dir ディレクトリ以下のファイルをすべて読み込んで, 一つのテキストにして返す。
    ファイルごとに、ファイル名を文頭にわかりやすく追加する。

    target_exts に指定した拡張子のファイルのみを対象とする。

    ignore_paths に指定したパスは無視する。
    ignore_paths には、ファイル名やディレクトリ名を指定する。
    アスタリスクを使ってワイルドカードを指定することもできる。

    Args:
        root_dir (str): ルートディレクトリ
        target_exts (list): 対象の拡張子のリスト
        ignore_paths (list): 無視するパスのリスト

    Returns:
        str: テキスト
    """
    import os
    import glob

    files = glob.glob(f'{root_dir}/**', recursive=True)
    files.sort()
    text = ''

    for file in files:
        if os.path.isdir(file):
            continue
        if target_exts and not any([file.endswith(ext) for ext in target_exts]):
            continue
        if any([glob.fnmatch.fnmatch(file, ignore_path) for ignore_path in ignore_paths]):
            continue
        with open(file, 'r') as f:
            text += '---\n'
            text += f'# {file}\n\n'
            text += f.read()
            text += '\n\n'

    print(f"文字列の長さ: {len(text)}")

    import tiktoken
    encoding = tiktoken.encoding_for_model("gpt-4-1106-preview")
    tokens = len(encoding.encode(text))
    print(f"トークン数: {tokens}\n")

    if tokens >= 120000:
        raise Exception('トークン数が120000を超えています。')

    return text

### generate_prompt

In [ ]:
def generate_prompt(root_dir: str, target_exts: list, ignore_paths: list, question: str):
    """
    LLM に入力するプロンプトを生成する

    Args:
        root_dir (str): ルートディレクトリ
        target_exts (list): 対象の拡張子のリスト
        ignore_paths (list): 無視するパスのリスト
        question (str): 質問

    Returns:
        str: プロンプト
    """
    prompt = f"""
    {get_knowledge_string(root_dir, target_exts, ignore_paths)}


    上記のテキストを参考に質問に答えてください。

    Q: {question}
    """
    return prompt

## メイン

In [ ]:
enabled = False

In [ ]:
root_dir = '/Users/kiarina/Documents/GitHub/buysell-technologies/gyro_infra/terraform'
target_exts = []
ignore_paths = ['*.terraform*', '*.git*']

question = """
staging 環境のリソースをリストアップしてください。
各リソースについて、簡単な説明もしてください。
"""

if enabled:
    from langchain.chat_models import ChatOpenAI
    chat = ChatOpenAI(model='gpt-4-1106-preview')
    print(chat.predict(generate_prompt(root_dir, target_exts, ignore_paths, question)))

In [ ]:
enabled = True